In [2]:
import os
from pathlib import Path

In [ ]:
# ollama pull llama3
# pip install langchain-nomic

In [1]:
from langsmith import traceable
from langchain_community.document_loaders import DirectoryLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
sntnc_trnsfrmr_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
output_dir = Path(r"D:\Ed\RAGs\Streamlit_RAG_1\docling\Manual fr RAG")
loader = DirectoryLoader(output_dir, glob="*.md")
docs = loader.load()
len(docs)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits_pypdf = text_splitter.split_documents(docs)
all_splits_pypdf_texts = [d.page_content for d in all_splits_pypdf]
baseline = Chroma.from_texts(
    texts=all_splits_pypdf_texts,
    collection_name="baseline",
    embedding=sntnc_trnsfrmr_embeddings,
)
retriever_baseline = baseline.as_retriever()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

sntnc_trnsfrmr_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

urls = [
    "https://example.com/post1",
    "https://example.com/post2",
    "https://example.com/post3",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [6]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from langchain_ollama import ChatOllama, OllamaLLM

llm_llama3 = ChatOllama(
    model="llama3.2:1b",
    format="json",
    temperature=0,
    # other params...
)

prompt = PromptTemplate(
    template="""system
You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals.
    Give a binary score 'yes' or 'no' to indicate whether the document is relevant to the question.
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    user
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n assistant""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm_llama3 | JsonOutputParser()

In [7]:
prompt = PromptTemplate(
    template="""system You are an expert at routing a user question to a vectorstore or web search. Use the vectorstore for questions on LLM agents, prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and no preamble or explanation. Question to route: {question} assistant""",
    input_variables=["question"],
)

question_router = prompt | llm_llama3 | JsonOutputParser()

In [8]:
prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. user
    Question: {question}
    Context: {context}
    Answer: assistant""",
    input_variables=["question", "context"],
)

rag_chain = prompt | llm_llama3 | JsonOutputParser()

In [9]:
prompt = PromptTemplate(
    template="""system You are a grader assessing whether an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. user
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation} assistant""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm_llama3 | JsonOutputParser()

In [ ]:
from langgraph.graph import END, StateGraph, START

class GraphState(TypedDict):
    question: str
    generation: str
    web_search: str
    documents: List[str]

workflow = StateGraph(GraphState)

workflow.add_node("websearch", web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

workflow.add_conditional_edges(
    START,
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

app = workflow.compile()

In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [17]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever_baseline,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts.",
)

tools = [tool]

In [14]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=retriever_baseline.invoke,
        description=(
            'use this tool when answering queries to get '
            'more information about the topic'
        )
    )
]

In [19]:
# Construct the ReAct agent
agent = create_react_agent(llm_llama3, tools, prompt)

In [22]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True,   max_iterations=2,
    stop_sequence=True  # Ensure the agent stops after the final answer)

In [23]:
agent_executor.invoke({"input": "what is the accuracy of CI1A J Thermocouple?"})



> Entering new AgentExecutor chain...
{}  Invalid Format: Missing 'Action:' after 'Thought:'

KeyboardInterrupt: 

In [24]:
# Required imports
from pydantic import BaseModel

class ActionOutput(BaseModel):
    action: str
    action_input: str


In [26]:
def get_prompt(user_input):
    return f"""
    User input: {user_input}
    Thought: you should always think about what to do
    Action: the action to take, should be one of the tools
    Action Input: the input to the action
    Observation: the result of the action
    ...
    Final Answer: summary of the final process or a prompt for the user
    """

In [28]:
from langchain_experimental.llms import LMFormatEnforcer

lm_format_enforcer = LMFormatEnforcer(
    json_schema=ActionOutput.schema(), pipeline=llm_llama3
)
results = lm_format_enforcer.predict(get_prompt("I want to sync Contact between Salesforce and ServiceNow"))
print(results)

AttributeError: 'ChatOllama' object has no attribute '_forward_params'

In [30]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

In [31]:
response_schemas = [
    ResponseSchema(name="Action", description="Action to be taken"),
    ResponseSchema(
        name="Thought",
        description="thought on what to do",
    ),
    ResponseSchema(name="Observation", description="result of action")
    # ResponseSchema(name="Observation", description="result of action")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [32]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [33]:
chain = prompt | llm_llama3 | output_parser

In [34]:
chain.invoke({"question": "what's the capital of france?"})

OutputParserException: Got invalid return object. Expected key `Action` to be present, but got {}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 